In [1]:
# short example 

input = '''R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2'''

In [2]:
import numpy as np

In [3]:
H = np.array([0, 0])
T = np.array([0, 0])
visited = {}

visited[np.array2string(T)] = 1

In [4]:
for line in input.split('\n'):
    direc = line.split()[0]
    steps = int(line.split()[1])

    for i in range(steps):
        if direc=="R":
            D = np.array([1, 0])
        if direc=="U":
            D = + np.array([0, 1])
        if direc=="L":
            D = + np.array([-1, 0])
        if direc=="D":
            D = + np.array([0, -1])
        H = H + D

        F = H - T
        Fsq = F[0]**2 + F[1]**2

        if Fsq >= 4:
            T = T + np.sign(F)
            if not np.array2string(T) in visited:
                visited[np.array2string(T)] = 1



In [5]:
visited

{'[0 0]': 1,
 '[1 0]': 1,
 '[2 0]': 1,
 '[3 0]': 1,
 '[4 1]': 1,
 '[4 2]': 1,
 '[4 3]': 1,
 '[3 4]': 1,
 '[2 4]': 1,
 '[3 3]': 1,
 '[3 2]': 1,
 '[2 2]': 1,
 '[1 2]': 1}

In [6]:
len(visited)

13

In [7]:
# longer example

input = '''R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20'''

In [8]:
rope = np.array([[0, 0] for i in range(10)])
visited = {}

visited[np.array2string(rope[9])] = 1

for line in input.split('\n'):
    print('Move:', line)
    direc = line.split()[0]
    steps = int(line.split()[1])

    for i in range(steps):
        if direc=="R":
            D = np.array([1, 0])
        if direc=="U":
            D = + np.array([0, 1])
        if direc=="L":
            D = + np.array([-1, 0])
        if direc=="D":
            D = + np.array([0, -1])
        rope[0] = rope[0] + D
        print('H:', rope[0])

        for k in range(1, 10):          
            F = rope[k-1] - rope[k]
            Fsq = F[0]**2 + F[1]**2
            if Fsq in [4, 5, 8]:
                rope[k] += np.sign(F)
            if k==9 and not np.array2string(rope[k]) in visited:
                visited[np.array2string(rope[k])] = 1
                    
            print('Knot', k, rope[k])

Move: R 5
H: [1 0]
Knot 1 [0 0]
Knot 2 [0 0]
Knot 3 [0 0]
Knot 4 [0 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [2 0]
Knot 1 [1 0]
Knot 2 [0 0]
Knot 3 [0 0]
Knot 4 [0 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [3 0]
Knot 1 [2 0]
Knot 2 [1 0]
Knot 3 [0 0]
Knot 4 [0 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [4 0]
Knot 1 [3 0]
Knot 2 [2 0]
Knot 3 [1 0]
Knot 4 [0 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [5 0]
Knot 1 [4 0]
Knot 2 [3 0]
Knot 3 [2 0]
Knot 4 [1 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
Move: U 8
H: [5 1]
Knot 1 [4 0]
Knot 2 [3 0]
Knot 3 [2 0]
Knot 4 [1 0]
Knot 5 [0 0]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [5 2]
Knot 1 [5 1]
Knot 2 [4 1]
Knot 3 [3 1]
Knot 4 [2 1]
Knot 5 [1 1]
Knot 6 [0 0]
Knot 7 [0 0]
Knot 8 [0 0]
Knot 9 [0 0]
H: [5 3]
Knot 1 [5 2]
Knot 2 [4 1]
Knot 3 [3 1]
Knot 4 [2 1]
Knot 5 [1 1]
Knot 6 [0 0]
Knot 7 [0 0

In [9]:
visited

{'[0 0]': 1,
 '[1 1]': 1,
 '[2 2]': 1,
 '[1 3]': 1,
 '[2 4]': 1,
 '[3 5]': 1,
 '[4 5]': 1,
 '[5 5]': 1,
 '[6 4]': 1,
 '[7 3]': 1,
 '[8 2]': 1,
 '[9 1]': 1,
 '[10  0]': 1,
 '[ 9 -1]': 1,
 '[ 8 -2]': 1,
 '[ 7 -3]': 1,
 '[ 6 -4]': 1,
 '[ 5 -5]': 1,
 '[ 4 -5]': 1,
 '[ 3 -5]': 1,
 '[ 2 -5]': 1,
 '[ 1 -5]': 1,
 '[ 0 -5]': 1,
 '[-1 -5]': 1,
 '[-2 -5]': 1,
 '[-3 -4]': 1,
 '[-4 -3]': 1,
 '[-5 -2]': 1,
 '[-6 -1]': 1,
 '[-7  0]': 1,
 '[-8  1]': 1,
 '[-9  2]': 1,
 '[-10   3]': 1,
 '[-11   4]': 1,
 '[-11   5]': 1,
 '[-11   6]': 1}

In [10]:
len(visited)

36